# ADMM 算法

### ADMM 算法

ADMM 可以用来求解形如
$$\begin{align*}
\min_{x,z}\  & f(x)+g(z)\\
\mathrm{s.t.}\  & Ax+Bz=c
\end{align*}$$
的优化问题，其中 $f$ 和 $g$ 是凸函数。

ADMM 的迭代公式为
$$
\begin{align*}
x^{k+1} & =\underset{x}{\arg\min}\ f(x)+\frac{\rho}{2}\Vert Ax+Bz^{k}-c+u^{k}\Vert^{2}\\
z^{k+1} & =\underset{z}{\arg\min}\ g(z)+\frac{\rho}{2}\Vert Ax^{k+1}+Bz-c+u^{k}\Vert^{2}\\
u^{k+1} & =u^{k}+Ax^{k+1}+Bz^{k+1}-c.
\end{align*}
$$

定义原问题残差 $r^{k+1}=Ax^{k+1}+Bz^{k+1}-c$ 和对偶问题残差 $s^{k+1}=\rho A'B(z^{k+1}-z^{k})$。当 $||r^k||$ 和 $||s^k||$ 小于某个阈值时即可认为算法收敛。

### Least Absolute Deviation (LAD)

LAD 是一种稳健回归方法，它与线性回归中的最小二乘方法（OLS）类似，但用残差的绝对值之和来替代平方和。为了与 ADMM 算法的记号匹配，我们用 $A\in\mathbb{R}^{n\times p}$ 表示自变量矩阵，$b\in\mathbb{R}^n$ 表示因变量向量，要估计的回归系数为 $x\in\mathbb{R}^p$。于是 LAD 的目标函数为 $$\Vert Ax-b\Vert_1,$$ 其中 $\Vert v\Vert_1$ 表示向量 $v=(v_1,\ldots,v_n)'$ 的 $L^1$ 范数，即 $\Vert v\Vert_1=|v_1|+\cdots+|v_n|$。

LAD 可以改写为 ADMM 的形式：$f(x)=0$，$g(z)=||z||_1$，$B=-I$，$c=b$。其迭代公式为

$$
\begin{align*}
x^{k+1} & =(A'A)^{-1}A'(b+z^{k}-u^{k})\\
z^{k+1} & =S_{1/\rho}(Ax^{k+1}-b+u^{k})\\
u^{k+1} & =u^{k}+Ax^{k+1}-z^{k+1}-b,
\end{align*}
$$

其中 $S_{\kappa}(a)$ 为 soft-thresholding 运算符，定义为

$$
S_{\kappa}(a)=\begin{cases}
a-\kappa, & a>\kappa\\
0, & |a|\le\kappa\\
a+\kappa, & a<-\kappa
\end{cases},
$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

相应地，原问题残差为 $r^{k+1}=Ax^{k+1}-z^{k+1}-b$，对偶问题残差为 $s^{k+1}=-\rho A'(z^{k+1}-z^{k})$。

### 利用 ADMM 求解 LAD

In [ ]:
import numpy as np
np.set_printoptions(linewidth=100)

先生成模拟数据：

In [ ]:
np.random.seed(123)
n = 1000
p = 10
A = np.random.normal(size=(n, p))
xtrue = np.random.normal(size=p)
b = A.dot(xtrue) + np.random.normal(size=n)

In [ ]:
xtrue

array([-1.24096967, -0.31294679, -0.84894679,  2.37795259,  0.65750062,  0.21308689, -0.49097031,
       -1.0815104 ,  0.00480111, -0.36079657])

设定 $x$、$z$ 和 $u$ 的初值：

In [ ]:
# 这里后续记得修改
x = np.zeros(p)
z = np.zeros(n)
u = np.zeros(n)

设定 $\rho$ 的取值（理论上可以是任意的正数）：

In [ ]:
rho = 1.0 # 实验，rho=10或者。。。

编写 soft-thresholding 函数，使其可以直接作用于向量：

In [ ]:
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)
    
soft_thresholding([-3, -2, -1, 0, 1, 2, 3], 1.5)

array([-1.5, -0.5, -0. ,  0. ,  0. ,  0.5,  1.5])

演示一步 $x$ 的迭代：

In [ ]:
xnew = np.linalg.solve(A.T.dot(A), A.T.dot(b + z - u)) # 不要写大型循环，
xnew

array([-1.24034079, -0.25873666, -0.90518866,  2.33812078,  0.69147325,  0.15743223, -0.4450978 ,
       -1.12812669, -0.02567582, -0.36984311])

演示一步 $z$ 的迭代：

In [ ]:
znew = soft_thresholding(A.dot(xnew) - b + u, 1.0 / rho) # 不要重复计算某个东西
znew[:10]

array([-0.        , -0.01952984,  0.        ,  0.73800732,  0.        , -0.        ,  0.87067037,
        0.72896027, -0.44397696, -0.        ])

演示一步 $u$ 的迭代：

In [ ]:
unew = u + A.dot(xnew) - znew - b 
# A.dot(xnew)上一步也有，不要重复计算
# A.dot(xnew) - znew - b 与后面计算rk相同 所以rk = unew - u
unew[:10]

array([-0.14818386, -1.        ,  0.37729366,  1.        ,  0.22272854, -0.99080063,  1.        ,
        1.        , -1.        , -0.38022739])

注意，在 $z$ 和 $u$ 的更新中都出现了 `A.dot(xnew)`，因此可以将其结果保存下来，避免重复计算。

计算原问题残差 $r^{k+1}=Ax^{k+1}-z^{k+1}-b$，其本质上就是 `unew - u`。

In [ ]:
resid_r = unew - u
np.linalg.norm(resid_r)

22.870132559316538

计算对偶问题残差 $s^{k+1}=-\rho A'(z^{k+1}-z^{k})$。

In [ ]:
resid_s = -rho * A.T.dot(znew - z)
np.linalg.norm(resid_s)

11.613498072547548

接下来将整个过程写入一个循环，同时设定最大迭代次数为10000，收敛的阈值为0.001。

In [ ]:
max_iter = 10000
tol = 0.001

for i in range(max_iter):
    # x 更新
    xnew = np.linalg.solve(A.T.dot(A), A.T.dot(b + z - u))
    # z 更新
    Axnew = A.dot(xnew)
    znew = soft_thresholding(Axnew - b + u, 1.0 / rho)
    # u 更新
    unew = u + Axnew - znew - b
    # 计算残差大小
    resid_r_norm = np.linalg.norm(unew - u)
    resid_s_norm = rho * np.linalg.norm(A.T.dot(znew - z))
    # 更新 x、z 和 u 的取值
    x = xnew
    z = znew
    u = unew
    # 打印残差信息，判断是否收敛
    if i % 100 == 0:
        print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
    if resid_r_norm <= tol and resid_s_norm <= tol:
        break

Iteration 0, ||r|| = 0.000115, ||s|| = 0.000975


In [ ]:
x

array([-1.19230848, -0.28642899, -0.89053513,  2.35251214,  0.66217182,  0.14198784, -0.43247972,
       -1.11299057, -0.01374415, -0.38485577])

In [ ]:
xtrue

array([-1.24096967, -0.31294679, -0.84894679,  2.37795259,  0.65750062,  0.21308689, -0.49097031,
       -1.0815104 ,  0.00480111, -0.36079657])

**注意：**注意到在每一次迭代中都要计算 $(A'A)^{-1}v$，其中 $v$ 是某个向量。如果直接使用 `np.linalg.solve()`，计算量会非常大。一种更好的方法是先对 $A'A$ 进行 Cholesky 分解（$A'A$ 是正定矩阵），然后再解线性方程组。

In [ ]:
from scipy.linalg import cho_factor, cho_solve

# 使用 Cholesky 求解线性方程组
c, lower = cho_factor(A.T.dot(A))
v1 = np.random.normal(size=p)
v2 = np.random.normal(size=p)
res1 = cho_solve((c, lower), v1)
res2 = cho_solve((c, lower), v2)

# 验证结果
print(res1)
print(np.linalg.solve(A.T.dot(A), v1))
print()
print(res2)
print(np.linalg.solve(A.T.dot(A), v2))

[-0.00062024 -0.00065258  0.00037566 -0.00177019  0.0011255   0.00068691 -0.00143911 -0.00131038
  0.00226992  0.00014166]
[-0.00062024 -0.00065258  0.00037566 -0.00177019  0.0011255   0.00068691 -0.00143911 -0.00131038
  0.00226992  0.00014166]

[ 0.0003871   0.00219593 -0.00098795 -0.00114172 -0.00035525 -0.0006054   0.00044722  0.00079882
  0.00227984  0.00074191]
[ 0.0003871   0.00219593 -0.00098795 -0.00114172 -0.00035525 -0.0006054   0.00044722  0.00079882
  0.00227984  0.00074191]


In [ ]:
max_iter = 10000
tol = 0.001

for i in range(max_iter):
    # x 更新
    xnew = cho_solve((c, lower), A.T.dot(b + z - u))
    # z 更新
    Axnew = A.dot(xnew)
    znew = soft_thresholding(Axnew - b + u, 1.0 / rho)
    # u 更新
    unew = u + Axnew - znew - b
    # 计算残差大小
    resid_r_norm = np.linalg.norm(unew - u)
    resid_s_norm = rho * np.linalg.norm(A.T.dot(znew - z))
    # 更新 x、z 和 u 的取值
    x = xnew
    z = znew
    u = unew
    # 打印残差信息，判断是否收敛
    if i % 100 == 0:
        print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
    if resid_r_norm <= tol and resid_s_norm <= tol:
        break

Iteration 0, ||r|| = 0.000116, ||s|| = 0.000961


In [ ]:
## 此处插入代码作答
def update_z(A, b, x, u, rho):
    # 插入代码
    a = A.dot(x) - b + u
    k= 1/rho
    znew = np.sign(a) * np.maximum(0.0, np.abs(a) - k)
    return znew

In [ ]:
## 此处插入代码作答
def update_x(A, b, z, u):
    # 插入代码
    xnew = np.linalg.solve(A.T.dot(A), A.T.dot(b+z-u))
    return xnew

In [ ]:
## 此处插入代码作答
from scipy.linalg import cho_factor, cho_solve

def admm_lad(A, b, rho=1.0, maxit=10000, eps=1e-3, verbose=0):
    ##初始化
    n, p = np.shape(A)
    x = np.zeros(p)
    z = np.zeros(n)
    u = np.zeros(n)
    ##使用Cholesky分解
    c, lower = cho_factor(A.T.dot(A))
    ##更新迭代
    for i in range(maxit):
        # x 更新
        #xnew = update_x(A, b, z, u)
        xnew = cho_solve((c,lower), A.T.dot(b+z-u))
        # z 更新
        znew = update_z(A, b, xnew, u, rho)
        # u 更新
        unew = u + A.dot(xnew) - znew - b
        # 计算残差大小
        resid_r_norm = np.linalg.norm(unew - u)
        resid_s_norm = rho * np.linalg.norm(A.T.dot(znew - z))
        # 更新 x、z 和 u 的取值
        x = xnew
        z = znew
        u = unew
        # 打印残差信息，判断是否收敛
        if i % 1000 == 0 and verbose > 0:
            print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
        if resid_r_norm <= eps and resid_s_norm <= eps:
            break
    return i+1, x
